In [308]:
#import libraries
import pandas as pd
import graphviz
import pydot

print(graphviz.__version__)
print(pydot.__version__)

0.14.1
1.4.1


In [309]:
#cleaning the file, we want to create a df that resembles df_2
#from the orig pokemon org csv, we are going to edit each record and insert it into a new df that will be 
#formatted like df_2
df = pd.read_csv("Pokemon_Org.csv")
df.head(5)

,ID,Name,Title,Reports To:
0,82,Magneton,Manager,NaN
1,51,Dugtrio,Assitant Manager 1,82.0
2,34,Nidoking,Assitant Manager 2,82.0
3,61,Poliwhirl,Assitant Manager 3,82.0
4,56,Mankey,Supervisor,82.0


In [310]:
#create subset on the higher ups ie Executive, Managers, Supervisors
first_level = df.loc[df['Title'] != "Staff", :]

In [311]:
#create staff subsets based on supervisors
mankey_56_staff = df.loc[df['Reports To:'] == 56, :]
kabuto_140_staff = df.loc[df['Reports To:'] == 140, :]
kakuna_14_staff = df.loc[df['Reports To:'] == 14, :]
rhydon_112_staff = df.loc[df['Reports To:'] == 112, :]

#how many employees are under each supervisor/managers?
print(mankey_56_staff.shape[0])
print(kabuto_140_staff.shape[0])
print(kakuna_14_staff.shape[0])
print(rhydon_112_staff.shape[0])

31
69
27
16


In [312]:
#We want a readable org chart(not too long) so we convert each supervisor staff lists to strings 
#that hold 25 names each
def employeeBoxLimit(name_list):
    '''This function returns a list with each element only holding 25 employees. If the list has 2 elements with names
    then the manager/supervisor will have 2 employee boxes associated with him/her
    Input: employee/staff list
    Output: a list with 3 elements(list), each element only holds 25 names'''
    num_name_boxes = []
    
    list_1 = []
    list_2 = []
    list_3 = []

    counter = 0
    for name in name_list:
        if counter <= 24:
            list_1.append(name)
        elif counter <= 49:
            list_2.append(name)
        else:
            list_3.append(name)

        counter += 1
      
    num_name_boxes.append(list_1)
    num_name_boxes.append(list_2)
    num_name_boxes.append(list_3)
    
    return num_name_boxes


In [313]:
#create the number of employee boxes associated with each supervisor
mankey_boxes = employeeBoxLimit(list(mankey_56_staff["Name"]))
kabuto_boxes = employeeBoxLimit(list(kabuto_140_staff["Name"]))
kakuna_boxes = employeeBoxLimit(list(kakuna_14_staff["Name"]))
rhydon_boxes = employeeBoxLimit(list(rhydon_112_staff["Name"]))

In [314]:
def createEmployeeList(a_list, seperator):
    '''This function returns a list of employees
    input: a list with 3 elements of list type, a list are employees associated with 1 supervisors
    output: a list with 3 elements with type str, list of employees converted to a string with a seperator'''
    name_list = []
    
    #del any empty elements in the list
    counter = 0
    for element in a_list:
        if len(element) == 0:
            a_list.pop(counter)
            
        counter += 1
    
    #iterate through each name and create a string with seperator
    for element in a_list:
        a_str = ""
        for name in element:
            info = name + seperator 
            a_str = a_str + info
        a_str = "Employees\l" + a_str #"\l" in graphviz, left aligns text
        name_list.append(a_str)
        
    return name_list

In [315]:
#create list of Employee strings for each Manager/Supervisor "\l" in graphviz, left aligns text
mankey_staff_list = createEmployeeList(mankey_boxes, "\l")
kabuto_staff_list = createEmployeeList(kabuto_boxes, "\l")
kakuna_staff_list = createEmployeeList(kakuna_boxes, "\l")
rhydon_staff_list = createEmployeeList(rhydon_boxes, "\l")

In [316]:
#create a new dataframe with only Managers & Supervisors
first_level_indiv = first_level.loc[first_level["Title"].isin(["Manager","Supervisor"]), :]

In [317]:
#create a list of the Managers and Supervisors, name + title, "\l" in graphviz, left aligns text
first_level_name_list = []

for index,row in first_level_indiv.iterrows():
    info = row["Name"] + "\l" + row["Title"] + "\l"
    first_level_name_list.append(info)

In [318]:
#iterate through the first level df and grab the relevant info (name, title)
first_level_names = []
for index, row in first_level.iterrows():
    info = row["Name"]
    first_level_names.append(info)
  
#convert the supporting staff to the Manager to 1 box that is under the Manager
support_str = ""
for name in first_level_names[1:4]:
    support_str = support_str + name + "\l"
support_str = "Staff\l" + support_str

In [319]:
#add the support staff to the first level name list at index 2
first_level_name_list.insert(1, support_str)
# print(first_level_name_list)

In [320]:
#create a list, every element as 2 items, name and reports to
#executive record
exe = [first_level_name_list[0], first_level_name_list[0]]

#support staff record
support = [first_level_name_list[1], first_level_name_list[0]]

#supervisor records
super_mankey = [first_level_name_list[2], first_level_name_list[0]]
super_kabuto = [first_level_name_list[3], first_level_name_list[0]]
super_kakuna = [first_level_name_list[4], first_level_name_list[0]]
super_rhydon = [first_level_name_list[5], first_level_name_list[0]]

#employee records
emp_mankey_1 = [mankey_staff_list[0], first_level_name_list[2]]
emp_mankey_2 = [mankey_staff_list[1], first_level_name_list[2]]
emp_kabuto_1 = [kabuto_staff_list[0], first_level_name_list[3]]
emp_kabuto_2 = [kabuto_staff_list[1], first_level_name_list[3]]
emp_kabuto_3 = [kabuto_staff_list[2], first_level_name_list[3]]
emp_kakuna_1 = [kakuna_staff_list[0], first_level_name_list[4]]
emp_kakuna_2 = [kakuna_staff_list[1], first_level_name_list[4]]
emp_rhydon_1 = [rhydon_staff_list[0], first_level_name_list[5]]

#here's the list of all the records we need
info_list = [exe, support, super_mankey, super_kabuto, super_kakuna, super_rhydon, emp_mankey_1, emp_mankey_2, 
             emp_kabuto_1, emp_kabuto_2, emp_kabuto_3, emp_kakuna_1, emp_kakuna_2, emp_rhydon_1]

In [321]:
#add all info to a df
column_names = ["Name", "Reports To:"]
df_1 = pd.DataFrame(info_list, columns=column_names)

#create an id column
df_1['ID'] = df_1.reset_index().index

In [322]:
def create_graph(df, title, executive_ID, staff_ID, manager_ID_start, manager_ID_end, chart_name):
    '''This function creates an hierarchal organization chart.
    Input: df = data frame, that is organized in a particular way, reference df_1 or df_2
           title = title of the org chart, str type
           executive_ID = the ID of the top person/boss/executive, int type
           staff_ID = the ID of the supporting staff of the executive, these people are not managers, int type
           manager_ID_start = there are many managers and they are consecutive located after one another in 
                              the data frame, the ID of the first manager ID, int type
           manager_ID_end = the ID of the last manager ID, int type
           chart_name = the chart image file name
    Output: an png file of an organization chart
    Note: You can change cell colors, refer to http://graphviz.org/doc/info/colors.html'''
    
    #import libraries
    import graphviz
    import pydot
    
    # instantiate a graph
    f = pydot.Dot(graph_type='graph', label = title , rankdir = 'BT', labelloc = "t")

    #Create Graph Nodes and interconnecting Edges
    for index, row in df.iterrows():

        #add nodes, based on ID, nodes will be different colors
        if row["ID"] == executive_ID:
            f.add_node(pydot.Node(str(row["Name"]), shape = 'box', fillcolor="lightsalmon", style = "filled"))
        elif row["ID"] == staff_ID:
            f.add_node(pydot.Node(str(row["Name"]), shape = 'box', fillcolor="lightsteelblue1", style = "filled"))
        elif manager_ID_start <= row["ID"] <= manager_ID_end:
            f.add_node(pydot.Node(str(row["Name"]), shape = 'box', fillcolor="cornsilk", style = "filled"))
        else:
            f.add_node(pydot.Node(str(row["Name"]), shape = 'box', fillcolor="gray84", style = "filled"))

        #draw edges(connections) between nodes
        if row["Name"] != row["Reports To:"]:
            f.add_edge(pydot.Edge(str(row["Name"]), str(row["Reports To:"]), dir = "back"))
        else:
            continue

    return f.write_png(chart_name)

In [323]:
#Example 1 : Cleaning the csv file in python and running the following code by passing a data frame
#uses df_1: the dataframe we just created from above

#Create the Org Chart
create_graph(df_1, 'Pokemon Org Chart\n', 0, 1, 2, 5, 'example1_graph.png')

In [324]:
#Example 2 : All you need to do is input the csv file, 
#uses pokemon_org_V2.csv, this csv is manually formated in excel

df_2 = pd.read_csv("Pokemon_Org_V2.csv")

#Create the Org Chart
create_graph(df_2, 'Pokemon Org Chart\n', 0, 1, 2, 5, 'example2_graph.png')